In [1]:
# Deleting files that we do not need, files generated from a previous run
import os
import glob

extension_list=["*inp*","*out","*ener","*rest*","*Hess*","*REST*","*.xyz","*.pdb"]
for name in extension_list:
    
    filelist=glob.glob(name)
    for file in filelist:
        os.remove(file)


# ### Loading modules

# In[2]:



import mbuild as mb 

import numpy as np

import unyt as u

from cp2kmdpy.molecule_optimization import Molecule_optimization
from cp2kmdpy.md import MD
#from cp2kmd.molecule_optimization import MD_main
import setter

In [2]:
!ls

DLT.ipynb	__pycache__	 sample_test.py  test_simple.py
Untitled.ipynb	md_pre_out_file  setter.py	 untitled


In [3]:

class Cl2(mb.Compound): # this class builds a chlorine molecule with a bond-length given in the chlorine2 x coor (nm)
    def __init__(self):
        super(Cl2, self).__init__()
        
        chlorine1= mb.Particle(pos=[0.0, 0.0, 0.0], name='Cl')
        chlorine2= mb.Particle(pos=[0.2, 0.0, 0.0], name='Cl')
        self.add([chlorine2,chlorine1])
        self.add_bond((chlorine2,chlorine1))
        
class FCl(mb.Compound): # this class builds a chlorine molecule with a bond-length given in the chlorine2 x coor (nm)
    def __init__(self):
        super(FCl, self).__init__()
        
        fluorine1= mb.Particle(pos=[0.0, 0.0, 0.0], name='F')
        chlorine2= mb.Particle(pos=[0.2, 0.0, 0.0], name='Cl')
        self.add([fluorine1,chlorine2])
        self.add_bond((fluorine1,chlorine2))


In [4]:
molecule=Cl2();
box=mb.box.Box(lengths=[1,1,1])

In [5]:
p=Molecule_optimization(molecule=molecule,box=box,cutoff=200,functional='PBE',periodicity='XYZ',n_iter=3)

In [6]:
p.optimization_initialization()

scf_tolerance not specified, set as 1e-6
basis_set_filename not defined, set as BASIS_MOLOPT
potential_filename not specified, set as GTH_POTENTIALS
fixed_list not specified, set as 1
You can change default settings in setter.single_molecule_opt_file


In [7]:
setter.single_molecule_opt_files(p)

p.run_optimization()

Initial structure saved as Cl2_unoptimized_coord.xyz
Molecule optimization file saved as Cl2_optimization_input.inp
Input file name given to runner is Cl2_optimization_input.inp
Output file name  is Cl2_optimization_output.out
Output position file name is Cl2_opt-pos-1.xyz


In [8]:
a=p.return_optimized_molecule()

In [9]:
a

<Cl2 2 particles, non-periodic, 1 bonds, id: 140175793524296>

In [10]:
q=MD(molecules=[a],box=box,cutoff=200,functional='PBE',basis_set={'Cl':'DZVP-MOLOPT-GTH'},periodicity='XYZ',n_molecules=2,traj_type='PDB',seed=1)


In [11]:
q.temperature=273.15*u.K
q.ensemble='NVT'

In [13]:
q.md_initialization()

project_name not specified, set as sample_project
scf_tolerance not specified, set as 1e-6
basis_set_filename not defined, set as BASIS_MOLOPT
potential_filename not specified, set as GTH_POTENTIALS
simulation_time not specified, set as 1 ps
time_step not specified, time_step set as 1 fs as the lighest element has mass 35.453 au
You can change default settings in setter.md_files


In [15]:
setter.md_files(q)

MD initial structure saved as sample_project.xyz
MD input file saved as sample_project_md_input.inp


In [16]:
!cat sample_project_md_input.inp

&GLOBAL
  RUN_TYPE     MD
  PROJECT      sample_project
  PRINT_LEVEL  LOW
  SEED  1
&END GLOBAL
&MOTION
  &MD
    ENSEMBLE        NVT
    TIMESTEP        1.0
    STEPS           1
    TEMPERATURE     273.15
    &THERMOSTAT       
    &END THERMOSTAT       
    &BAROSTAT       
    &END BAROSTAT       
    &AVERAGES       
    &END AVERAGES       
    &PRINT       
      &ENERGY       
        &EACH       
        &END EACH       
      &END ENERGY       
      &PROGRAM_RUN_INFO       
        &EACH       
        &END EACH       
      &END PROGRAM_RUN_INFO     
    &END PRINT       
  &END MD
 
  &CONSTRAINT
    &FIXED_ATOMS      
    &END FIXED_ATOMS       
  &END CONSTRAINT
 
  &PRINT 
    &FORCES        OFF
      &EACH       
      &END EACH       
    &END FORCES        
    &RESTART_HISTORY        ON
      &EACH       
        MD              500             
      &END EACH       
    &END RESTART_HISTORY        
    &RESTART        ON
      BACKUP_COPIES        3
      &EACH  

In [ ]:
!pwd